In [1]:
import pandas as pd
import numpy as np

This notebook will :

Process all raw data 

NeST 

L1000 

Viral infections

* Keep only homo sapiens

* For perturbations keep same cell line and time for consitency and fairness
* 3-100 genes
* get a total of 300 sets
* get test data 10 from each 

In [44]:
genesets_colnames =["Source", "GeneSetID", "GeneSetName", "GeneList", "n_Genes"]
# Create an empty DataFrame with specified column names
genesets_DF = pd.DataFrame(columns=genesets_colnames)

# Convert all columns to string type
genesets_DF = genesets_DF.astype(str)

# Convert 'n_Genes' column to integer type
genesets_DF['n_Genes'] = genesets_DF['n_Genes'].astype(int)

# Make a copy of the DataFrame to represent an empty DataFrame
genesets_empty_DF = genesets_DF.copy()

genesets_DF, genesets_empty_DF

(Empty DataFrame
 Columns: [Source, GeneSetID, GeneSetName, GeneList, n_Genes]
 Index: [],
 Empty DataFrame
 Columns: [Source, GeneSetID, GeneSetName, GeneList, n_Genes]
 Index: [])

### NeST

In [3]:
import pandas as pd

# Read the delimited file
NeST_DF = pd.read_csv("data/Omics_data/NeST__IAS_clixo_hidef_Nov17.edges", delimiter="\t", header=None)

# Rename columns for clarity (assuming columns X1, X2, X3 based on the R code)
NeST_DF.columns = ['X1', 'X2', 'X3']

# Filter rows where X3 is "gene"
NeST_DF = NeST_DF[NeST_DF['X3'] == 'gene']

# Group by X1 and create a concatenated list of X2 values
NeST_DF = NeST_DF.groupby('X1')['X2'].apply(lambda x: ' '.join(x)).reset_index()

# Rename the column to GeneList
NeST_DF.rename(columns={'X2': 'GeneList'}, inplace=True)

# Calculate the number of genes
NeST_DF['n_Genes'] = NeST_DF['GeneList'].apply(lambda x: len(x.split()))

# Filter rows where n_Genes is between 3 and 100
NeST_DF = NeST_DF[(NeST_DF['n_Genes'] >= 3) & (NeST_DF['n_Genes'] <= 100)]

# Add Source, GeneSetID, and GeneSetName columns
NeST_DF['Source'] = 'NeST'
NeST_DF['GeneSetID'] = NeST_DF['X1']
NeST_DF['GeneSetName'] = NeST_DF['X1']

# Select relevant columns and reorder them according to genesets_colnames
genesets_colnames = ['GeneSetID', 'GeneSetName', 'GeneList', 'n_Genes', 'Source']  # replace with actual column names
NeST_DF = NeST_DF[genesets_colnames]

NeST_DF.head()

,GeneSetID,GeneSetName,GeneList,n_Genes,Source
3,Cluster1-10,Cluster1-10,CTRL HSD17B14 KIAA0232 PAQR8 PLA2G1B RNF145 SG...,12,NeST
4,Cluster1-11,Cluster1-11,LMF1 MFHAS1 MR1 PLA2G1B RASL11A RNF145 SLC2A6 ...,12,NeST
5,Cluster1-12,Cluster1-12,AMY2B CNPY2 EGFL7 LDLR LPL LRP8 LRPAP1 MYLIP P...,12,NeST
6,Cluster1-13,Cluster1-13,ACO1 CDKAL1 CDX2 CIAO1 CITED2 FAM96A FAM96B FB...,16,NeST
7,Cluster1-14,Cluster1-14,DTWD1 DTWD2 POLD2 POLD4 POLH POLI RAD18 REV1 R...,15,NeST


In [4]:
NeST_DF.shape

(2153, 5)

### L1000

https://maayanlab.cloud/Harmonizome/dataset/LINCS+L1000+CMAP+Signatures+of+Differentially+Expressed+Genes+for+Small+Molecules

In [17]:
import pandas as pd

# Read the delimited file
L1000_all_DF = pd.read_csv("data/Omics_data/L1000__gene_attribute_edges.txt", delimiter="\t", skiprows=1)

# Group by the specified column and create a concatenated list of GeneSym values
L1000_all_DF = L1000_all_DF.groupby('Perturbation ID_Perturbagen_Cell Line_Time_Time Unit_Dose_Dose Unit')['GeneSym'].apply(lambda x: ' '.join(x)).reset_index()

# Rename the column to GeneList
L1000_all_DF.rename(columns={'GeneSym': 'GeneList'}, inplace=True)

# Calculate the number of genes
L1000_all_DF['n_Genes'] = L1000_all_DF['GeneList'].apply(lambda x: len(x.split()))

# Filter rows where n_Genes is between 3 and 100
L1000_all_DF = L1000_all_DF[(L1000_all_DF['n_Genes'] >= 3) & (L1000_all_DF['n_Genes'] <= 100)]

# Add Source, GeneSetID, and GeneSetName columns
L1000_all_DF['Source'] = 'L1000'
L1000_all_DF['GeneSetID'] = L1000_all_DF['Perturbation ID_Perturbagen_Cell Line_Time_Time Unit_Dose_Dose Unit']
L1000_all_DF['GeneSetName'] = L1000_all_DF['Perturbation ID_Perturbagen_Cell Line_Time_Time Unit_Dose_Dose Unit']

# Split the GeneSetName into separate columns
split_columns = L1000_all_DF['GeneSetName'].str.split('_', expand=True)
L1000_all_DF[['Drug', 'Reagent', 'Cellline', 'duration', 'duration_unit', 'dosage', 'dosage_unit']] = split_columns.iloc[:, :7]

# Remove the original groupby column
L1000_all_DF.drop(columns=['Perturbation ID_Perturbagen_Cell Line_Time_Time Unit_Dose_Dose Unit'], inplace=True)


L1000_all_DF.head()

,GeneList,n_Genes,Source,GeneSetID,GeneSetName,Drug,Reagent,Cellline,duration,duration_unit,dosage,dosage_unit
0,1060P11.3 ARMCX1 BTG2 CA2 CASP10 CD74 CITED2 C...,64,L1000,BRD-A00267231_HEMADO_A549_24.0_h_10.0_um,BRD-A00267231_HEMADO_A549_24.0_h_10.0_um,BRD-A00267231,HEMADO,A549,24.0,h,10.0,um
4,AQP3 CD36 CKAP4 IGFBP5 PTGS2 RNASE1 TRIB2 ADD3...,18,L1000,BRD-A00267231_HEMADO_PC3_24.0_h_10.0_um,BRD-A00267231_HEMADO_PC3_24.0_h_10.0_um,BRD-A00267231,HEMADO,PC3,24.0,h,10.0,um
5,1060P11.3 A2M AQP3 ATP1B1 AUTS2 CA12 CALM3 CAP...,64,L1000,BRD-A00267231_HEMADO_PC3_6.0_h_10.0_um,BRD-A00267231_HEMADO_PC3_6.0_h_10.0_um,BRD-A00267231,HEMADO,PC3,6.0,h,10.0,um
6,1060P11.3 ADRB2 ARHGEF2 ASS1 ATF3 ATF4 CACNA1A...,53,L1000,BRD-A00420644_-666_A375_6.0_h_10.0_um,BRD-A00420644_-666_A375_6.0_h_10.0_um,BRD-A00420644,-666,A375,6.0,h,10.0,um
9,1060P11.3 ABHD4 AKR1B10 ARF4 CAMK2N1 CITED2 CX...,65,L1000,BRD-A00420644_-666_HEPG2_6.0_h_10.0_um,BRD-A00420644_-666_HEPG2_6.0_h_10.0_um,BRD-A00420644,-666,HEPG2,6.0,h,10.0,um


In [18]:
# Filter the DataFrame for specific Cellline, duration, and dosage
L1000_DF = L1000_all_DF[(L1000_all_DF['Cellline'] == 'MCF7') & (L1000_all_DF['duration'] == '6.0') & (L1000_all_DF['dosage'] == '10.0')]

# Select the desired columns
genesets_colnames = ['GeneList', 'n_Genes', 'Source', 'GeneSetID', 'GeneSetName']
L1000_DF = L1000_DF[genesets_colnames]
L1000_DF.head()

,GeneList,n_Genes,Source,GeneSetID,GeneSetName
15,CITED2 COL5A1 CRABP2 KCTD12 MDFIC MMP2 NRP1 OR...,24,L1000,BRD-A00546892_-666_MCF7_6.0_h_10.0_um,BRD-A00546892_-666_MCF7_6.0_h_10.0_um
20,1060P11.3 ADM AHR AMIGO2 ARL4C ATP10D CAV2 CD4...,47,L1000,BRD-A00993607_ALPRENOLOL_MCF7_6.0_h_10.0_um,BRD-A00993607_ALPRENOLOL_MCF7_6.0_h_10.0_um
22,ABAT ASS1 CHI3L1 CHST2 CLDN3 EIF5B FRZB GAL HE...,59,L1000,BRD-A00993607_Alprenolol hydrochloride_MCF7_6....,BRD-A00993607_Alprenolol hydrochloride_MCF7_6....
29,AMIGO2 AREG GAS6 GPR37 IFT57 PELI1 SQLE AKAP12...,9,L1000,BRD-A01320529_Salmeterol_MCF7_6.0_h_10.0_um,BRD-A01320529_Salmeterol_MCF7_6.0_h_10.0_um
39,1060P11.3 CPE EFNB2 HIST1H2AC IL1R2 INHBB LYPD...,15,L1000,BRD-A01346607_FLUMETHASONE_MCF7_6.0_h_10.0_um,BRD-A01346607_FLUMETHASONE_MCF7_6.0_h_10.0_um


In [19]:
L1000_DF.shape

(994, 5)

### Viral Infections

In [21]:
viral_Infections_all_long_DF

,#,icSARA deltaORF6_30Hour,icSARA deltaORF6_36Hour,icSARA deltaORF6_54Hour,icSARA deltaORF6_60Hour,icSARA deltaORF6_48Hour,icSARA deltaORF6_72Hour,icSARA deltaORF6_24Hour,icSARS CoV_24Hour,icSARS CoV_60Hour,...,Ebolavirus(ZEBOV)_3day_Liver,Ebolavirus(ZEBOV)_5day_Liver,Norwalk Virus_HG23,A-CA-04-2009(H1N1)_0Hour.3,SARS-BatSRBD_0Hour.2,SARS-CoV_0Hour.2,SARS-ddORF6_0Hour,HCoV-EMC2012_0Hour,A-CA-04-2009(H1N1)_0Hour.4,A-Netherlands-602-2009(H1N1)_0Hour
1,ANP32AP1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
2,OR52N5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
3,ZNF699,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
4,ZNF705D,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
5,XAGE2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
21409,SSX5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,-0.173978,0.051522,-0.161991,-0.089695,-0.756813,-0.093471,-0.002526,-0.115508
21410,LOC100507397,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
21411,PCDHA13,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
21412,LOC391322,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000


In [42]:
import pandas as pd
import numpy as np

# Read the viral infection file 
viral_Infections_all_long_DF = pd.read_csv("data/Omics_data/Viral_Infections_gene_attribute_matrix_standardized.txt", delimiter="\t", skiprows=1, low_memory=False)

# Rename the column '#' to 'GeneSym'
viral_Infections_all_long_DF.rename(columns={'#': 'GeneSym'}, inplace=True)

# Select all columns except the second and third
viral_Infections_all_long_DF.drop(viral_Infections_all_long_DF.columns[[1, 2]], axis=1, inplace=True)

# Remove the first row
viral_Infections_all_long_DF = viral_Infections_all_long_DF.iloc[1:, :]

# Pivot the DataFrame to a long format
viral_Infections_all_long_DF = viral_Infections_all_long_DF.melt(id_vars=['GeneSym'], var_name='name', value_name='value')

# Subset the DataFrame where abs(value) >= 2
viral_Infections_all_long_DF = viral_Infections_all_long_DF[np.abs(viral_Infections_all_long_DF['value']) >= 2]

# Group by 'name' and create a concatenated list of GeneSym values
viral_Infections_all_long_DF = viral_Infections_all_long_DF.groupby('name')['GeneSym'].apply(lambda x: ' '.join(x)).reset_index()

# Rename the column to GeneList
viral_Infections_all_long_DF.rename(columns={'GeneSym': 'GeneList'}, inplace=True)

# Calculate the number of genes
viral_Infections_all_long_DF['n_Genes'] = viral_Infections_all_long_DF['GeneList'].apply(lambda x: len(x.split()))

# Filter rows where n_Genes is between 3 and 100
viral_Infections_all_long_DF = viral_Infections_all_long_DF[(viral_Infections_all_long_DF['n_Genes'] >= 3) & (viral_Infections_all_long_DF['n_Genes'] <= 100)]

# Add Source, GeneSetID, and GeneSetName columns
viral_Infections_all_long_DF['Source'] = 'Viral_Infections'
viral_Infections_all_long_DF['GeneSetID'] = viral_Infections_all_long_DF['name']
viral_Infections_all_long_DF['GeneSetName'] = viral_Infections_all_long_DF['name']

# Split the GeneSetID into separate columns
split_columns = viral_Infections_all_long_DF['GeneSetID'].str.split('_', expand=True)
viral_Infections_all_long_DF[['virus', 'duration']] = split_columns.iloc[:, :2]

# Define the desired columns to select
genesets_colnames = ['GeneList', 'n_Genes', 'Source', 'GeneSetID', 'GeneSetName']

# Select the desired columns
viral_Infections_all_long_DF = viral_Infections_all_long_DF[genesets_colnames]

viral_Infections_all_long_DF.head()

,GeneList,n_Genes,Source,GeneSetID,GeneSetName
4,GGT8P ATP6AP1 UROC1 DNAJC7 ERVH-6 LOC643355 AD...,85,Viral_Infections,A-CA-04-2009(H1N1)_0Hour.2,A-CA-04-2009(H1N1)_0Hour.2
9,LOC554207 OSCAR ZNF709 PHACTR2 CENPO RWDD2B PD...,65,Viral_Infections,A-CA-04-2009(H1N1)_12Hour.2,A-CA-04-2009(H1N1)_12Hour.2
23,MIS12 MRE11A GALNT7 USP14 RASA4 NAA20 RNF149 T...,84,Viral_Infections,A-CA-04-2009(H1N1)_36Hour.2,A-CA-04-2009(H1N1)_36Hour.2
25,CALD1 C9ORF53 PCDHB18 AMPH C19ORF67 SPAM1 NLGN...,60,Viral_Infections,A-CA-04-2009(H1N1)_3Hour,A-CA-04-2009(H1N1)_3Hour
44,CGA RESP18 CX3CR1 RBP2 ZNF788 ZDHHC22 GABRB1 E...,93,Viral_Infections,A-Netherlands-602-2009(H1N1)_3Hour,A-Netherlands-602-2009(H1N1)_3Hour


In [45]:
# Append the first 126 rows of NeST_DF
genesets_DF = pd.concat([genesets_empty_DF, NeST_DF.head(126)], ignore_index=True)

# Append the first 126 rows of L1000_DF
genesets_DF = pd.concat([genesets_DF, L1000_DF.head(126)], ignore_index=True)

# Append the first 48 rows of viral_Infections_all_long_DF
genesets_DF = pd.concat([genesets_DF,  viral_Infections_all_long_DF.head(48)], ignore_index=True)

print(genesets_DF.shape)

(300, 5)


In [46]:
# Extract the 'Source' column
source_column = genesets_DF['Source']
source_column.value_counts()


NeST                126
L1000               126
Viral_Infections     48
Name: Source, dtype: int64

In [47]:
genesets_DF

,Source,GeneSetID,GeneSetName,GeneList,n_Genes
0,NeST,Cluster1-10,Cluster1-10,CTRL HSD17B14 KIAA0232 PAQR8 PLA2G1B RNF145 SG...,12
1,NeST,Cluster1-11,Cluster1-11,LMF1 MFHAS1 MR1 PLA2G1B RASL11A RNF145 SLC2A6 ...,12
2,NeST,Cluster1-12,Cluster1-12,AMY2B CNPY2 EGFL7 LDLR LPL LRP8 LRPAP1 MYLIP P...,12
3,NeST,Cluster1-13,Cluster1-13,ACO1 CDKAL1 CDX2 CIAO1 CITED2 FAM96A FAM96B FB...,16
4,NeST,Cluster1-14,Cluster1-14,DTWD1 DTWD2 POLD2 POLD4 POLH POLI RAD18 REV1 R...,15
...,...,...,...,...,...
295,Viral_Infections,hMPV_72Hour,hMPV_72Hour,RNMT SAMD9L CEBPB CEBPG PSMD12 RSRC2 XBP1 GADD...,22
296,Viral_Infections,icSARA deltaORF6_48Hour,icSARA deltaORF6_48Hour,FGF19 ADM2 LRIT1 UCHL1 SLC19A1 TSSC1 INPP5J HY...,49
297,Viral_Infections,icSARS CoV_12Hour.1,icSARS CoV_12Hour.1,TSPY3 LGALS14 PPBPP2 LARS MRPS10 PAX9 RBM22 LO...,16
298,Viral_Infections,icSARS CoV_3Hour,icSARS CoV_3Hour,ANK2 GAB3 TGFB3 CECR3 MUC19 LOC100507053 ZNF36...,99


In [20]:
genesets_DF.to_csv("data/omics_revamped.txt", sep="\t", index=False)